In [ ]:
import pandas as pd
import numpy as np

## Loading Data

In [ ]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v20.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v20.csv")
submission_set = pd.read_csv("./data/final_submission_set.csv")

## Predictive Model Learning

In [ ]:
cat_names = ['callsign',
            'adep', 
            'ades', 
            'aircraft_type', 
            'wtc', 
            'airline',
            'offblock_hour',
            'offblock_minute', 
            'offblock_day_of_week',
            'offblock_weekday_name',
            'offblock_month',
            'offblock_week_of_year', 
            'offblock_season', 
            'arrival_hour',
            'arrival_minute',
            'arrival_season',
            'arrival_weekday_name',
            'is_offblock_weekend',
            'is_offblock_rush_hour',
            'flight_duration_category',                       
            'adep_region', 
            'ades_region', 
            'same_country_flight',
            'same_region_flight',                        
            'flight_direction',
            'is_intercontinental',
            'Manufacturer',
            'Model_FAA',
            'Physical_Class_Engine',
            'FAA_Weight',
            'adep_geo_cluster',
            'ades_geo_cluster']

In [ ]:
dataset = pd.concat([challenge_set_updated, submission_set_updated], axis=0)

In [ ]:
pd.set_option('display.max_rows', None)

print(dataset.isnull().mean().sort_values(ascending=False))

pd.reset_option('display.max_rows')

In [ ]:
# Dropping columns with too many NaNs
threshold = 0.4
df = dataset.dropna(thresh=int((1 - threshold) * len(dataset)), axis=1)

In [ ]:
# Imputation of NaNs
columns_with_nan = dataset.isna().any()
for col in dataset.columns[columns_with_nan]:
    dataset.loc[:, col] = dataset.fillna(dataset[col].median())

In [ ]:
df = dataset.iloc[0:challenge_set_updated.shape[0], :]

In [ ]:
X = df.drop('tow', axis=1)
y = df.tow

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from lightgbm.basic import LightGBMError

# Specify your categorical columns
categorical_cols = [
    'callsign', 'adep', 'ades', 'aircraft_type', 'wtc', 'airline',
    'offblock_season', 'offblock_weekday_name', 'arrival_season', 'arrival_weekday_name',
    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction',
    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight'
]

# Convert categorical columns to type 'category'
for col in categorical_cols:
    if col in X.columns:
        X[col] = X[col].astype('category')

# Splitting the dataset
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBM parameters
params = {
    'learning_rate': 0.01,
    'reg_lambda': 0.46415888336127775,
    'reg_alpha': 0.166810053720005,
    'min_child_weight': 4,
    'max_depth': 13,
    'colsample_bytree': 0.6,
    'objective': 'regression',
    'random_state': 42,
    'n_estimators': 50000,
    'metric': 'rmse',
    'n_jobs': -1,
    'device': 'gpu',  # Enable GPU usage
    'subsample': 1.0
}

# Creating LightGBM datasets with free_raw_data set to False
train_data = lgb.Dataset(X_train, label=y_train, 
                         categorical_feature=categorical_cols, 
                         free_raw_data=False)
val_data = lgb.Dataset(X_val, label=y_val, 
                       categorical_feature=categorical_cols, 
                       free_raw_data=False)

# Training the LightGBM model
try:
    model = lgb.train(
        params,
        train_data,
        valid_sets=[val_data],
        valid_names=['valid'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=20),
            lgb.log_evaluation(period=100)  # Logs evaluation every 100 rounds
        ]
    )
except LightGBMError as e:
    # Handle GPU error, switch to CPU
    print("Switching to CPU due to GPU limitation.")
    params['device'] = 'cpu'  # Switch device to CPU
    model = lgb.train(
        params,
        train_data,
        num_boost_round=5000,
        valid_sets=[val_data],
        valid_names=['valid'],
        callbacks=[
            lgb.early_stopping(stopping_rounds=20),
            lgb.log_evaluation(period=100)  # Logs evaluation every 100 rounds
        ]
    )


In [ ]:
# Predict on the validation set
y_pred = model.predict(X_val)

# Save predictions to a CSV file
pd.DataFrame(data={'tow': y_pred}).to_csv('lightgbm_val_v20.csv', index=False)
print("Predictions saved to 'lightgbm_val_v20.csv'")

In [ ]:
df_test = dataset.iloc[challenge_set_updated.shape[0]:, :]
df_test.head()

In [ ]:
X_test = df_test.drop('tow', axis=1)


In [ ]:
# Assuming X_test is your test dataset

# Convert categorical columns to 'category' type with the same categories as in X_train
for col in categorical_cols:
    if col in X_test.columns:
        X_test[col] = pd.Categorical(X_test[col], categories=X_train[col].cat.categories)

# Now you can safely use the model for prediction
y_pred = model.predict(X_test)
y_test = y_pred

In [ ]:
dft0 = pd.read_csv('./data/final_submission_set.csv')
dft0['tow'] = y_test
dft0[['flight_id', 'tow']].to_csv('submissions/lightligbm.csv', index=False)